In [ ]:
import torch
# Library utama PyTorch untuk tensor dan autograd

import torchvision
# Library vision PyTorch (di sini belum digunakan, tapi sering dipakai untuk dataset/gambar)

from torch.utils.data import Dataset, DataLoader
# Dataset: class dasar untuk membuat dataset custom
# DataLoader: untuk batching, shuffle, dan loading data otomatis

import numpy as np
# NumPy untuk load dan manipulasi data numerik

import math
# Library matematika (di kode ini tidak digunakan secara eksplisit)

In [ ]:
class WineDataset(Dataset):
    # Membuat dataset custom dengan mewarisi torch.utils.data.Dataset

    def __init__(self, transform=None):
        # Constructor dataset
        # transform: fungsi preprocessing yang akan diterapkan ke setiap sample

        xy = np.loadtxt(
            '/content/drive/MyDrive/Triad/Kuliah/Machine Learning/src/wine.csv',
            delimiter=',',
            dtype=np.float32,
            skiprows=1
        )
        # Membaca file CSV:
        # - delimiter ',' → pemisah kolom
        # - dtype float32 → tipe data numerik
        # - skiprows=1 → melewati header

        self.x = xy[:, 1:]
        # Mengambil kolom ke-1 sampai akhir sebagai fitur (features)

        self.y = xy[:, [0]]
        # Mengambil kolom pertama sebagai label (target)
        # [0] ditulis sebagai [[0]] agar bentuknya (n_samples, 1)

        self.n_samples = xy.shape[0]
        # Menyimpan jumlah total sampel

        self.transform = transform
        # Menyimpan transform (jika ada) untuk digunakan di __getitem__


    def __getitem__(self, index):
        # Mengambil satu sampel berdasarkan index

        sample = self.x[index], self.y[index]
        # sample berupa tuple: (features, label)

        if self.transform:
            # Jika transform diberikan,
            # maka transform diterapkan ke sample
            sample = self.transform(sample)

        return sample
        # Mengembalikan sample yang sudah (atau belum) ditransform


    def __len__(self):
        # Mengembalikan jumlah total data
        return self.n_samples


class ToTensor:
    # Transform custom untuk mengubah NumPy array menjadi PyTorch Tensor

    def __call__(self, sample):
        # __call__ memungkinkan object dipanggil seperti fungsi

        input, targets = sample
        # Memisahkan fitur dan label

        return torch.from_numpy(input), torch.from_numpy(targets)
        # Mengubah NumPy → Tensor


class MulTransform:
    # Transform custom untuk mengalikan fitur dengan faktor tertentu

    def __init__(self, factor):
        self.factor = factor
        # Faktor pengali disimpan

    def __call__(self, sample):
        inputs, target = sample
        # Memisahkan fitur dan label

        inputs *= self.factor
        # Mengalikan fitur dengan faktor

        return inputs, target
        # Label tidak diubah


print('Without Transform')
# Dataset tanpa transform

dataset = WineDataset()
first_data = dataset[0]
# Mengambil satu sampel pertama

features, labels = first_data
# Memisahkan fitur dan label

print(type(features), type(labels))
# Menampilkan tipe data (NumPy array)

print(features, labels)
# Menampilkan isi data


print('\nWith Tensor Transform')
# Dataset dengan transform ToTensor

dataset = WineDataset(transform=ToTensor())
first_data = dataset[0]

features, labels = first_data

print(type(features), type(labels))
# Sekarang tipe data adalah torch.Tensor

print(features, labels)


print('\nWith Tensor and Multiplication Transform')
# Dataset dengan beberapa transform sekaligus

composed = torchvision.transforms.Compose([
    ToTensor(),
    MulTransform(4)
])
# Compose menggabungkan beberapa transform
# Urutan penting: ToTensor → lalu MulTransform

dataset = WineDataset(transform=composed)
first_data = dataset[0]

features, labels = first_data

print(type(features), type(labels))
# Tetap tensor

print(features, labels)
# Nilai fitur sudah dikalikan 4

print(type(features), type(labels))


Without Transform
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03] [1.]

With Tensor Transform
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) tensor([1.])

With Tensor and Multiplication Transform
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([5.6920e+01, 6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01,
        1.2240e+01, 1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01,
        4.2600e+03]) tensor([1.])
<class 'torch.Tensor'> <class 'torch.Tensor'>
